In [7]:
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, assemble, transpile
from qiskit.providers.ibmq import least_busy
from qiskit.circuit import Parameter 


import numpy as np
import pandas as pd
import plotly.express as px

In [12]:
theta_param, phi_param = Parameter('theta_param'), Parameter('phi_param') 

qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.ry(theta_param, qreg_q[0])
circuit.rz(phi_param, qreg_q[0])

circuit.rx(-np.pi/2, qreg_q[0])
circuit.sdg(qreg_q[1])
circuit.h(qreg_q[1])
circuit.tdg(qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.t(qreg_q[1])
circuit.h(qreg_q[1])
circuit.s(qreg_q[1])
circuit.cx(qreg_q[1], qreg_q[0])
circuit.rx(np.pi/2, qreg_q[0])
circuit.rx(np.pi/2, qreg_q[1])

circuit.rz(-phi_param, qreg_q[0])
circuit.ry(-theta_param, qreg_q[1])
circuit.measure(qreg_q[0], creg_c[0])
circuit.measure(qreg_q[1], creg_c[1])

In [13]:
N_shots=1024
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-01-04 19:05:51,095: Credentials are already in use. The existing account in the session will be replaced.


In [23]:
num_pts = 100

coords=[]

coords.append((0,0))
for i in range(int(num_pts/2)-1):
    coords.append((np.pi/(num_pts/2)*(i+1),0))
    coords.append((np.pi/(num_pts/2)*(i+1),-np.pi))
coords.append((np.pi,0))

x=[np.cos(point[1])*np.sin(point[0]) for point in coords]
y=[np.sin(point[1])*np.sin(point[0]) for point in coords]
z=[np.cos(point[0]) for point in coords]

#x, y, z = np.cos(phi) * np.sin(theta), np.sin(phi) * np.sin(theta), np.cos(theta);

# Plot the surface.
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, template="plotly_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()
target_points=np.array(coords)


## Santiago

In [28]:
backend = provider.backends.ibmq_santiago
id_file = 'eco_qcm_ids_santiago_1.txt'
result_txt='eco_results_ibm_santiago_1.txt'
data_sheet_file="eco_data_sheet_ibm_santiago_1.xlsx"

In [32]:
read_job_ids = open(id_file).read().splitlines()
read_job_ids = list(set(read_job_ids))
counts_array=[]
results_probabilities=[]
retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
for job in retrieved_jobs:
    counts_array=counts_array+job.result().get_counts()
for job in counts_array:
    #test both the copies
    prob0_qubit_0=job["00"]/N_shots+job["01"]/N_shots
    prob1_qubit_0=job["10"]/N_shots+job["11"]/N_shots
    prob0_qubit_1=job["00"]/N_shots+job["10"]/N_shots
    prob1_qubit_1=job["01"]/N_shots+job["11"]/N_shots
    results_probabilities.append([prob0_qubit_0,prob1_qubit_0,prob0_qubit_1,prob1_qubit_1])
    
results_probabilities=np.array(results_probabilities)

data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])

data_sheet.to_excel(data_sheet_file)

In [34]:
#FIDELITY OF FIRST COPY
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_0, template="plotly_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_0))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_0))

Number of points:  100
Average fidelity:  0.655380859375
Standard deviation:  0.23140751994812897


## Athens

In [35]:
backend = provider.backends.ibmq_athens
id_file = 'eco_qcm_ids_athens_1.txt'
result_txt='eco_results_ibm_athens_1.txt'
data_sheet_file="eco_data_sheet_ibm_athens_1.xlsx"

In [37]:
read_job_ids = open(id_file).read().splitlines()
read_job_ids = list(set(read_job_ids))
counts_array=[]
results_probabilities=[]
retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
for job in retrieved_jobs:
    counts_array=counts_array+job.result().get_counts()
for job in counts_array:
    #test both the copies
    prob0_qubit_0=job["00"]/N_shots+job["01"]/N_shots
    prob1_qubit_0=job["10"]/N_shots+job["11"]/N_shots
    prob0_qubit_1=job["00"]/N_shots+job["10"]/N_shots
    prob1_qubit_1=job["01"]/N_shots+job["11"]/N_shots
    results_probabilities.append([prob0_qubit_0,prob1_qubit_0,prob0_qubit_1,prob1_qubit_1])
    
results_probabilities=np.array(results_probabilities)

data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])

In [38]:
data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])

In [39]:
#FIDELITY OF FIRST COPY
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_0, template="plotly_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_0))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_0))

Number of points:  100
Average fidelity:  0.680283203125
Standard deviation:  0.23335241991271455


## Vigo

In [40]:
backend = provider.backends.ibmq_vigo
id_file = 'eco_qcm_ids_vigo_1.txt'
result_txt='eco_results_ibm_vigo_1.txt'
data_sheet_file="eco_data_sheet_ibm_vigo_1.xlsx"

In [42]:
read_job_ids = open(id_file).read().splitlines()
read_job_ids = list(set(read_job_ids))
counts_array=[]
results_probabilities=[]
retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
for job in retrieved_jobs:
    counts_array=counts_array+job.result().get_counts()
for job in counts_array:
    #test both the copies
    prob0_qubit_0=job["00"]/N_shots+job["01"]/N_shots
    prob1_qubit_0=job["10"]/N_shots+job["11"]/N_shots
    prob0_qubit_1=job["00"]/N_shots+job["10"]/N_shots
    prob1_qubit_1=job["01"]/N_shots+job["11"]/N_shots
    results_probabilities.append([prob0_qubit_0,prob1_qubit_0,prob0_qubit_1,prob1_qubit_1])
    
results_probabilities=np.array(results_probabilities)

data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])

In [43]:
#FIDELITY OF FIRST COPY
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_0, template="plotly_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_0))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_0))

Number of points:  100
Average fidelity:  0.688466796875
Standard deviation:  0.21541673836131964


## Ourense

In [44]:
backend = provider.backends.ibmq_ourense
id_file = 'eco_qcm_ids_ourense_1.txt'
result_txt='eco_results_ibm_ourense_1.txt'
data_sheet_file="eco_data_sheet_ibm_ourense_1.xlsx"

In [47]:
read_job_ids = open(id_file).read().splitlines()
read_job_ids = list(set(read_job_ids))
counts_array=[]
results_probabilities=[]
retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
for job in retrieved_jobs:
    counts_array=counts_array+job.result().get_counts()
for job in counts_array:
    #test both the copies
    prob0_qubit_0=job["00"]/N_shots+job["01"]/N_shots
    prob1_qubit_0=job["10"]/N_shots+job["11"]/N_shots
    prob0_qubit_1=job["00"]/N_shots+job["10"]/N_shots
    prob1_qubit_1=job["01"]/N_shots+job["11"]/N_shots
    results_probabilities.append([prob0_qubit_0,prob1_qubit_0,prob0_qubit_1,prob1_qubit_1])
    
results_probabilities=np.array(results_probabilities)

data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])

In [48]:
#FIDELITY OF FIRST COPY
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_0, template="plotly_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_0))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_0))

Number of points:  100
Average fidelity:  0.719580078125
Standard deviation:  0.22734112295618925


## Valencia

In [49]:
backend = provider.backends.ibmq_valencia
id_file = 'eco_qcm_ids_valencia_1.txt'
result_txt='eco_results_ibm_valencia_1.txt'
data_sheet_file="eco_data_sheet_ibm_valencia_1.xlsx"

In [51]:
read_job_ids = open(id_file).read().splitlines()
read_job_ids = list(set(read_job_ids))
counts_array=[]
results_probabilities=[]
retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
for job in retrieved_jobs:
    counts_array=counts_array+job.result().get_counts()
for job in counts_array:
    #test both the copies
    prob0_qubit_0=job["00"]/N_shots+job["01"]/N_shots
    prob1_qubit_0=job["10"]/N_shots+job["11"]/N_shots
    prob0_qubit_1=job["00"]/N_shots+job["10"]/N_shots
    prob1_qubit_1=job["01"]/N_shots+job["11"]/N_shots
    results_probabilities.append([prob0_qubit_0,prob1_qubit_0,prob0_qubit_1,prob1_qubit_1])
    
results_probabilities=np.array(results_probabilities)

data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])

In [61]:
data_sheet.to_excel(data_sheet_file)

In [52]:
#FIDELITY OF FIRST COPY
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_0, template="plotly_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_0))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_0))

Number of points:  100
Average fidelity:  0.699345703125
Standard deviation:  0.18946020519751663


## Yorktown

In [53]:
backend = provider.backends.ibmqx2
id_file = 'eco_qcm_ids_yorktown_1.txt'
result_txt='eco_results_ibm_yorktown_1.txt'
data_sheet_file="eco_data_sheet_ibm_yorktown_1.xlsx"

In [55]:
read_job_ids = open(id_file).read().splitlines()
read_job_ids = list(set(read_job_ids))
counts_array=[]
results_probabilities=[]
retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
for job in retrieved_jobs:
    counts_array=counts_array+job.result().get_counts()
for job in counts_array:
    #test both the copies
    prob0_qubit_0=job["00"]/N_shots+job["01"]/N_shots
    prob1_qubit_0=job["10"]/N_shots+job["11"]/N_shots
    prob0_qubit_1=job["00"]/N_shots+job["10"]/N_shots
    prob1_qubit_1=job["01"]/N_shots+job["11"]/N_shots
    results_probabilities.append([prob0_qubit_0,prob1_qubit_0,prob0_qubit_1,prob1_qubit_1])
    
results_probabilities=np.array(results_probabilities)

data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])

In [56]:
#FIDELITY OF FIRST COPY
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_0, template="plotly_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_0))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_0))

Number of points:  100
Average fidelity:  0.640068359375
Standard deviation:  0.2278517774256846


## Melbourne

In [57]:
backend = provider.backends.ibmq_16_melbourne
id_file = 'eco_qcm_ids_melbourne_1.txt'
result_txt='eco_results_ibm_melbourne_1.txt'
data_sheet_file="eco_data_sheet_ibm_melbournen_1.xlsx"

In [59]:
read_job_ids = open(id_file).read().splitlines()
read_job_ids = list(set(read_job_ids))
counts_array=[]
results_probabilities=[]
retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
for job in retrieved_jobs:
    counts_array=counts_array+job.result().get_counts()
for job in counts_array:
    #test both the copies
    prob0_qubit_0=job["00"]/N_shots+job["01"]/N_shots
    prob1_qubit_0=job["10"]/N_shots+job["11"]/N_shots
    prob0_qubit_1=job["00"]/N_shots+job["10"]/N_shots
    prob1_qubit_1=job["01"]/N_shots+job["11"]/N_shots
    results_probabilities.append([prob0_qubit_0,prob1_qubit_0,prob0_qubit_1,prob1_qubit_1])
    
results_probabilities=np.array(results_probabilities)

data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])

In [60]:
#FIDELITY OF FIRST COPY
# Make data.
thetas=data_sheet["θ"]
phis=data_sheet["ϕ"]
x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

# Plot the surface. F_measured/F_theory
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_0, template="plotly_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()
print("Number of points: ", len(x))
print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_0))
print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_0))

Number of points:  100
Average fidelity:  0.68431640625
Standard deviation:  0.1974502452517484


## Execution

In [58]:
runningjobs=[]
job_ids=[]
index=0

while index*75 < num_pts:
    if((index+1)*75<num_pts):
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param : points[1]}) 
         for points in coords[index*75:(index+1)*75]]
        qobj = assemble(transpile(circuits, backend=backend), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open(id_file, 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    else:
        circuits = [circuit.bind_parameters(
        {theta_param: points[0], phi_param: points[1]}) 
         for points in coords[index*75:]]
        qobj = assemble(transpile(circuits, backend=backend), backend=backend, shots=N_shots)
        runningjobs.append(backend.run(qobj))
        job_ids.append(runningjobs[-1].job_id())
        with open(id_file, 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
            print("Maximum number of jobs reached. Waiting...")
            try:
                job_result = runningjobs[0].result()  # It will block until the job finishes.
                print("The job finished with result {}".format(job_result))
                runningjobs.pop(0)
            except JobError as ex:
                print("Something wrong happened!: {}".format(ex))
    with open(id_file, 'a') as file:
        file.write(runningjobs[-1].job_id()+"\n")
    index=index+1